In [8]:
DATASET_NAME = "football-players-detection-3zvbc"
DATASET_ROOT_FOLDER = os.path.join("data",DATASET_NAME)
MODEL_NAME = "sportimization-1"
MODEL_ARCH = "v9-s"
NUM_EPOCH = 1
BATCH_SIZE = 4
IMAGE_SIZE = 1280
RESUME = True
OUTPUT_PATH = "runs"#"/content/drive/MyDrive/"+MODEL_NAME #runs

In [9]:
import roboflow
import os

roboflow.login()

rf = roboflow.Roboflow()
project = rf.workspace("roboflow-jvuqo").project(DATASET_NAME)
dataset = project.version(12).download(model_format="yolov9",location=os.path.join(".","data",DATASET_NAME))


You are already logged into Roboflow. To make a different login,run roboflow.login(force=True).
loading Roboflow workspace...
loading Roboflow project...


In [10]:
# directory structure bullshit....

import shutil
import glob

image_dir = os.path.join(DATASET_ROOT_FOLDER,"images")
labels_dir = os.path.join(DATASET_ROOT_FOLDER,"labels")
phases = ["train","valid","test"]
for phase in phases:
    os.makedirs(os.path.join(image_dir,phase),exist_ok=True)
    imgs_to_move = glob.glob(os.path.join(DATASET_ROOT_FOLDER,phase,"images","*.jpg"))
    for img in imgs_to_move:
        shutil.move(img,os.path.join(image_dir,phase))
    os.makedirs(os.path.join(labels_dir,phase),exist_ok=True)
    labels_to_move = glob.glob(os.path.join(DATASET_ROOT_FOLDER,phase,"labels","*.txt"))
    for label in labels_to_move:
        shutil.move(label,os.path.join(labels_dir,phase))

In [11]:
# modify yolo config file
import yaml



with open("yolo/config/config.yaml","r") as file:
    data = yaml.safe_load(file)
    data["name"] = MODEL_NAME
    data["resume"] = RESUME
    defaults = data["defaults"]
    for item in defaults:
        if isinstance(item, dict):
            if "model" in item.keys():
                item["model"] = MODEL_ARCH
            elif "dataset" in item.keys():
                item["dataset"] = DATASET_NAME
            elif "task" in item.keys():
                item["task"] = "train-"+MODEL_NAME
    defaults[-1] = "general-"+MODEL_NAME
    hydra = data["hydra"]
    run = hydra["run"]
    run["dir"] = OUTPUT_PATH


        
    print(data)    
    with open(os.path.join("yolo","config",MODEL_NAME+".yaml"),"w") as out:
        yaml.safe_dump(data=data,stream=out,sort_keys=False)

with open("yolo/config/general.yaml","r") as file:
    data = yaml.safe_load(file)
    data["use_wandb"] = False
    data["use_tensorboard"] = False
    #data["cpu_num"] = 8
    data["image_size"] = [IMAGE_SIZE,IMAGE_SIZE]
    data["out_path"] = OUTPUT_PATH
    with open(os.path.join("yolo","config","general-"+MODEL_NAME+".yaml"),"w") as out:
        yaml.safe_dump(data=data,stream=out,sort_keys=False)

with open("yolo/config/task/train.yaml","r") as file:
    data = yaml.safe_load(file)
    data["epoch"] = NUM_EPOCH
    d = data["data"]
    d["batch_size"]= BATCH_SIZE
    with open(os.path.join("yolo","config","task","train-"+MODEL_NAME+".yaml"),"w") as out:
        yaml.safe_dump(data=data,stream=out,sort_keys=False)

with open(os.path.join("data",DATASET_NAME,"data.yaml"),"r") as file:
    data = yaml.safe_load(file)
    data["path"] = os.path.join("data",DATASET_NAME)
    data["train"] = "train"
    data["validation"] = "valid"
    data["test"] = "test"
    data["class_num"] = data["nc"]
    data.pop("nc")
    data.pop("val")
    with open(os.path.join("yolo","config","dataset",DATASET_NAME+".yaml"),"w") as out:
        yaml.safe_dump(data=data,stream=out,sort_keys=False)


{'hydra': {'run': {'dir': 'runs'}}, 'name': 'sportimization-1', 'defaults': ['_self_', {'task': 'train-sportimization-1'}, {'dataset': 'football-players-detection-3zvbc'}, {'model': 'v9-s'}, 'general-sportimization-1'], 'resume': True}


In [ ]:
# start training
